In [4]:
pip install qiskit numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 23.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.9/25.9 MB 22.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 18.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 6.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, AncillaRegister, Parameter
from qiskit.quantum_info import Statevector, Operator

import numpy as np
import SudoquCircuitBuilder as sudoqu
import PuzzleConfigurations as pc

# Summary of the algorithm:

Step 1: input a $2^2\times 2^2$ Sudoku board

Step 2: prepare input for quantum circuit initialization:
    - Find all possible configurations for each row, each column, and each $2\times 2$-quadrant of the given board. If a row/column/quadrant trivially only has one possible configuration, we immediately fill the unknows with that configuration. 
    - The number of rows + columns + quadrants is $4+4+4=12$. This is the number of constraints we will have. Each row/column/quadrant has to take on one of the possible configurations we have found in the previous step. 

Steo 3: Initialize qubits:
    - If $n$ is the number of unknowns left, then we need $n\times 2$ qubits to represent the unkowns. We initialize them as $H\ket{00}$.
    - For each constraint, we will need one ancillar qubit, which tells us if that constraint is satisfied or not.

Step 4: #todo




We begin by inputting the entries of a $2^2\times 2^2$ Sudoku game. Our convention will be to indicate unknown values by $0.$ All other values must be 1, 2, 3, 4.

In [ ]:
#todo: code that reads in a sudoky board

Now we will find all configurations for rows/columns/quadrants and complete the values which can be trivially filled.

In [3]:
puzzle = [
        [3, 1, 4, 2],
        [2, 0, 0, 1],
        [1, 0, 0, 0],
        [4, 0, 0, 3]
    ] #todo: this runs a predefined sudoku. Need to change that to plugging in the one read in above

print("Started with the following puzzle:")
for row in puzzle:
    print(row)
print()

pc.complete_all_trivial_missing_values(puzzle)

print("Completed trivial missing values. Current puzzle:")
for row in puzzle:
    print(row)
print()


Started with the following puzzle:
[3, 1, 4, 2]
[2, 0, 0, 1]
[1, 0, 0, 0]
[4, 0, 0, 3]

Completed trivial missing values. Current puzzle:
[3, 1, 4, 2]
[2, 4, 3, 1]
[1, 0, 0, 4]
[4, 0, 0, 3]



Our next step is to initiate our qubits and a circuit. Let $n$ be the number of unknowns on the board after completing obvous unknowns. To represent the four numbers 1, 2, 3, 4 we will need 2 bits. Each bit we can then represent via a quibit in the obvious way: $0\mapsto \ket{0}$ and $1\mapsto \ket{1}$. Thus we need two qubits to represent the four numbers 1, 2, 3, 4 via qubits. Hence, we initiate $n\times 2$ qubits for the unknowns. 

Notation: We will use the number $0$ to represent an unknown on the board. To save space, we will then represent the numbers 1, 2, 3, 4 as follows via bits: $$1\mapsto 00,$$
$$2\mapsto 01,$$
$$3\mapsto 10,$$ 
$$4\mapsto 11.$$

Notation: Let $i$ be an integer from $0$ to $n-1$. We denote by u_i the qubit corresponding to the zeroth bit of the $i$-th unknown and by t_i the first bit of the $i$-th unknown. For example, if the $7$-th unknown ends up being 3, we represent it with the bits $10$ and thus u_7=0 and t_7=1.